# Applied Data Science Capstone

## Table of Contents

* [Introduction](#Introduction)
* [Data](#Data)
* [Methodology](#Methodology)
* [Results](#Results)
* [Discussion](#Discussion)
* [Conclusion](#Conclusion)

# Introduction

## Business Problem

In this study I consider myself to be the owner of Fusion Coffee Lab, a successful, brick-and-mortar coffee shop near Arizona State University in Tempe, AZ and seek to find a neighborhood into which I can expand in the Denver, CO metropolitan area.

Combining demographic data with location data from a Location-Based Social Network (LBSN) I will attempt to use K-Means Clustering to find similar locations in the Denver metropolitan area that I'm hoping will allow me to limit my search areas and focus instead on a more specific location and available real-estate.

Knowing my current customers, my expansion would ideally be in an area in or near one of Denver's older but growing neighborhoods, possibly in or near one of the [Top Neighborhoods for Young Professionals](http://urbanphenix.com/neighborhoods-in-denver-for-young-professionals/), not in close proximity to an international airport (which is something I don't like about my current location), have less than 50% of existing coffee shops be independent, and possibly near some or all of the following 'anchor' venues:

* A university and/or medical complex
* An arts district and/or performing arts complex
* Professional office buildings


## Target Audience

While this study is limited in scope, namely the expansion of a single business, and may not have broad appeal, it does provide a working example and demonstrates the power of data in providing actionable intelligence to a business. If I were the business owner, it would help me to narrow my search to specific areas of Denver that might feel familiar and in which I could find similar customers, and help me to focus my efforts on other aspects of expansion, such as staffing, real-estate, and a modified business plan.

# Data

In order to solve the stated problem I am going to be combining data from multiple sources.

* A list of target zip codes in the Denver metropolitan area and for Tempe, AZ
* Demographic information for all the zip codes
* Categorical, venue and attraction data from an LBSN
* A comprehensive list of franchise coffee shops

The following data sources will be used to collect the necessary data

* Zip-Codes.com for the list of zip codes in and around Denver
* HomeTownLocator.com for the demographics of the various zip codes and for Tempe, AZ
* Google Geolocation API for locating the centroids of the zip codes
* Foursquare API for the recommended venue data and category data
* Wikipedia for a list of franchise coffee shops

## Data Acquisition

Note that the bulk of the code responsible for much of the work in this study, including obtaining the data, exists in my [**Python Utility Module**](https://github.com/jrfoster/Coursera_Capstone/blob/master/capstoneutils.py), which is also linked at the bottom of the study.

In [ ]:
# Import the utility library with the bulk of the code and then the rest of
# the libraries necessary for the remainder of the work.
import capstoneutils as csutil
import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import folium

from IPython.display import Image

# Set the Seaborn plot style
sns.set_style("whitegrid")

# Save a latex representation for all our outputs
pd.options.display.latex.repr=True

### Zip Codes and Demographics

The list of zip codes for Denver metro area are first obtained from Zip-Codes.com and to that list we add the zip code for Tempe, AZ (85281). This list is then used to scrape demographics from HometownLocator.com. Note that there are six zip codes in the Denver metro area for which HometownLocator has no demographics, and two for which it has demographics but the values are all zeros. When manually inspecting the pages for these zip codes, the site suggests to use `80202` as a substitute. In scraping demographics, I do fall back to that zip which creates duplicates in the results, which I then remove. This means that there are eight zip codes that are being removed from the list obtained via Zip-Codes.com.

Because of the growth in Denver and the associated inflation in the cost of real estate and therefore salaries, note that I am making an adjustment two two demographic features for Tempe. Specifically, I'm adjusting the median home value by a factor of 1.53 and the average household income by a factor of 1.21. This is based on information from [Sperling's Best Places](https://www.bestplaces.net/cost-of-living/tempe-az/denver-co/50000). These adjustments allow for a like-for-like comparison on the two features.

The demographics site provides 19 features per zip code which are divided into four categories: Population, Housing, Income and Households. The following table summarizes the features in each category.

| Section | Notes |
|------------|-------------------------------------------------------------------------------------|
| Population | Total population, population in families, households, density and diversity index |
| Housing | Total Housing Units (owner- and renter-occupied, vacant) and average  home values |
| Income | Median and mean household income and per capita income |
| Households | Total households, average household size, family households and average family size |

The following table displays a sample of the demographic data obtained.

In [ ]:
# Use the utility library to load the demographics either from a serialized dataframe
# or from the zip code, demographics and geocoding sources from scratch.
results = csutil.load_demographics()

# Adjust based on Sperling
results.loc[results.ZipCode == '85281', 'Median Home Value'] = int(results.loc[results.ZipCode == '85281', 'Median Home Value'] * 1.53)
results.loc[results.ZipCode == '85281', 'Average Household Income'] = int(results.loc[results.ZipCode == '85281', 'Average Household Income'] * 1.21)

# Show sample data from the resulting demographics
results[-5:].loc[:, ['ZipCode', 'Latitude', 'Longitude', 'Diversity Index3','Median Home Value', 'Population Density2']]

### Foursquare Venues

Next, the top 100 recommended venues are obtained for each of the zip codes using the Foursquare API. Note that we allow the Foursquare API to determine the appropriate radius for our search, because it knows the density of venues in the zip code and will choose an appropriate one to accommodate our request for 100 venues. 

The following table displays a sample of the venue data.

In [ ]:
# Use the utility library to load the Foursquare venue data either from a 
# serialized dataframe or from the Foursquare API from scratch. Afterwards
# display some sample data
result_venues = csutil.load_foursquare_venues(names = results['ZipCode'],lats = results['Latitude'],lngs=results['Longitude'])
result_venues[0:5].loc[:, ['ZipCode', 'Venue', 'Venue Main Category','Venue Top-Level Category']]


*Simpson's Diversity Index*

The method used in the course labs for clustering based on the top venues and their categories generated a lot of features. Clustering high-dimensional data comes with its own set of problems and usually requires techniques other than K-Means. In our case, this is true because the concept of 'distance' between points is based on the means of a lot of zeros and ones, which over a large number of features, becomes almost meaningless. In my own clustering work for this class, based on silhouette analysis, I discovered repeatedly that the best number of clusters was 2.

Because of this, I'm using what is called [Simpson's Diversity Index](http://www.countrysideinfo.co.uk/simpsons.htm) (SDI) which is normally meant for measuring biodiversity, but it does allow us to measure the *richness* and *evenness* of populations. It is easily adaptable to venues and their categories. In my approach, I'm collecting the tree of [Foursquare Categories](https://developer.foursquare.com/docs/resources/categories) and using the top-level categories as a sort of 'species'. This approach allows me to preserve the information in the types and numbers of venues and significantly reduce the dimensions in the dataset, and results in just 28 features in the final dataset.

The following table displays a sample of the SDI data.

In [ ]:
# Calculate our version of Simpson's Diversity Index. Note that we are using
# the Reciprocal Index.
t1 = result_venues.loc[:, ['ZipCode', 'Venue Top-Level Category', 'Venue Main Category', 'Venue']]
t2 = t1.groupby(by=['ZipCode', 'Venue Top-Level Category', 'Venue Main Category'])['Venue'].count().reset_index()
t2.rename(columns={'Venue': 'Little-n'}, inplace=True)
t3 = t2.groupby(by=['ZipCode', 'Venue Top-Level Category'])['Little-n'].sum().reset_index()
t3.rename(columns={'Little-n': 'Big-N'}, inplace=True)
t4 = pd.merge(t2,t3,left_on=['ZipCode','Venue Top-Level Category'], right_on=['ZipCode','Venue Top-Level Category'], how='inner', validate='m:1')
t4['Squared n-N Ratio'] = t4.apply(lambda row: pd.Series((row['Little-n'] / row['Big-N'])**2), axis=1)
t5 = t4.groupby(by=['ZipCode', 'Venue Top-Level Category'])['Squared n-N Ratio'].sum().reset_index()
t5['Simpsons RI'] = t5.apply(lambda row: pd.Series(1 / row['Squared n-N Ratio']), axis=1)
t5.drop('Squared n-N Ratio', axis=1, inplace=True)
t6 = pd.pivot_table(t5, index='ZipCode', columns='Venue Top-Level Category', values='Simpsons RI', fill_value=0.0)

# Merge our diversity indexes with the demographics for the final merged dataset
# We should end up with a shape of (87,31)
final_merged = pd.merge(results, t6, how='inner', left_on='ZipCode', right_on='ZipCode', indicator=False, validate='one_to_one')
final_merged[-5:].loc[:, ['ZipCode', 'Food', 'Nightlife Spot','Outdoors & Recreation', 'Shop & Service']]


*One-Hot Encoding*

For the last dataset we will utilize one-hot-encoding to generate dummy variables for the myriad of venue categories returned form Foursquare and then use those categories to generate the top ten most recommended venues for each of the target zip codes. This data will be used later in the study when we examine the similarity of the clusters we generate. 

The following table displays a sample of the top venues returned from Foursquare for each zip code.

In [ ]:
result_venues_sorted = csutil.get_top_venues(result_venues, num_top_venues=10)
result_venues_sorted[-5:].loc[:, ['ZipCode', '1st Most Common Venue', '2nd Most Common Venue','3rd Most Common Venue']]

# Methodology

Given the primary sources of data, I will be performing the following top-level steps in this study

1. Perform exploratory analysis of the main data sources to identify any patterns and to help with possible feature reduction
2. Perform K-Means Clustering to identify candidate zip codes, which are those that are clustered with Tempe
3. Explore the cluster containing Tempe, and identify the best zip codes, based on the stated criteria
4. Provide guidance on the top zip codes, based on the mix of franchise vs. independent coffee shops in the candidates

## Exploratory Analysis

Some exploratory analysis of the data is performed to identify any patterns or issues in the data that must be addressed prior to finding similar zip codes. 

### Denver Metro Area Zip Codes

First, lets visualize the zip codes in the Denver metro area being considered to get a sense for the area of coverage under consideration.

In [ ]:
# Locate Denver and make a base map of it
denxy = csutil.get_place_latlon('Denver, CO, USA')
map_den = folium.Map(location=denxy, zoom_start=10)

# Add circle markers for each zip code and draw the map
for zipcode, lat, lon in zip(results['ZipCode'], results['Latitude'], results['Longitude']):
    label = '{}'.format(zipcode)
    label = folium.Popup(label)
    folium.CircleMarker(
        location=[lat, lon],
        popup=label,
        radius=5,
        color='royalblue',
        weight=2,
        fill_color='cornflowerblue',
        fill_opacity=0.67).add_to(map_den)
    
# Save the map so we can include it in the PDF
csutil.save_map(map_den, html_fn='den_map.html', png_fn='./maps/den_map.png')

In [ ]:
# Display the map.
map_den

### Tempe

Below we examine both the top ten venues in Tempe and the rank of each of the demographic attributes.

In [ ]:
# Filter columns we are interested in for later use
filtered_columns = ['Venue', 'Venue Main Category', 'Venue Latitude', 'Venue Longitude']
tempe_venues = result_venues.loc[result_venues[result_venues.ZipCode == '85281'].index, filtered_columns]

# Get the counts of venues by category and choose the top 10 from the list. We
# have to sort these twice because of what I think is a bug in barh where you
# can't specify the order the bars appear in.
cats = tempe_venues.groupby(['Venue Main Category'], as_index=False).size().to_frame('count')
cats.sort_values(by=['count'], ascending=False, inplace=True)
cats.reset_index(inplace=True)
cats_top10 = cats[:10].sort_values(by=['count'], ascending=False)

# Get the numerical ranks for all the demographics features
demog = results.loc[:, "Average Family Size":"Vacant Housing Units"]
ranks = demog.rank(axis=0, ascending=False)
ranks['ZipCode'] = results['ZipCode']
td = ranks.loc[results['ZipCode'] == '85281', 
               "Average Family Size":"Vacant Housing Units"].reset_index(drop=True).transpose().reset_index()
td.columns = ['Measure','Numerical Rank']

# Draw the plots
index = np.arange(10)
fig, axs = plt.subplots(1,2,figsize=(24,12))
fig.subplots_adjust(wspace=.3)
sns.barplot(y="Venue Main Category", x="count", data=cats_top10, color='cornflowerblue', ax=axs[0])
axs[0].set_title('Top Ten Venue Categories in Tempe, AZ (85281)', fontdict={'size': 16, 'weight': 'bold'})
sns.barplot(x='Numerical Rank', y='Measure', data=td, color='sienna', ax=axs[1])
axs[1].set_title('Demographics Rankings for Tempe, AZ (85281)', fontdict={'size': 16, 'weight': 'bold'});

One other item of interest is the top venues visited after visiting my shop. The following code utilizes Foursquare's `NEXTVENUES` endpoint to find the top five venues most likely to be visited after my shop.

In [ ]:
next_venues = csutil.get_nextvenues()
next_venues.head()

### Demographics

As we have seen from displays of the dataset, the demographic data contain a great deal of variety in terms of features. Specifically the data contain

* Monetary values for things like income and property values, which may be very different in some cases
* A Diversity Index, which is essentially a probability that two persons, chosen at random from the zip code, belong to different race or ethnic groups. This is essentially another SDI.
* Quantitative values, such as the number of owner-occupied housing units and various population counts
* Measures such as population density which is the total population per square mile

Next I want to look at the descriptive statistics that summarize the central tendency, dispersion and shape of a dataset’s distribution, and to see if there are values that look like outliers. The following boxplots display some of the dataset's features, and are separated into groups of 'like' features.

In [ ]:
# Since there is so much variety in the data, I'm showing plots of related features
# so that the scales make sense
# Slice into separate dataframes
df_home = demog.loc[:, ['Average Home Value','Median Home Value']]
df_income = demog.loc[:, ['Average Household Income', 'Median Household Income', 'Per Capita Income']]
df_housing = demog.loc[:, ['Family Households', 'Owner Occupied HU', 'Renter Occupied HU', 'Total Households']]
df_population = demog.loc[:, ['Population Density2', 'Population in Familes', 'Population in Households','Total Population']]

# Create and display the plots
fig, axs = plt.subplots(2,2, figsize=(24,16))
fig.subplots_adjust(wspace=.3)
sns.boxplot(data=df_home, orient="h", palette="Set2", ax=axs[0,0], showmeans=True)
axs[0,0].set_title('Boxplot of Home Value Features', fontdict={'size': 18, 'weight': 'bold'})
sns.boxplot(data=df_income, orient="h", palette="Set2", ax=axs[0,1], showmeans=True)
axs[0,1].set_title('Boxplot of Income Features', fontdict={'size': 18, 'weight': 'bold'})
sns.boxplot(data=df_housing, orient="h", palette="Set2", ax=axs[1,0], showmeans=True)
axs[1,0].set_title('Boxplot of Housing Features', fontdict={'size': 18, 'weight': 'bold'})
sns.boxplot(data=df_population, orient="h", palette="Set2", ax=axs[1,1], showmeans=True)
axs[1,1].set_title('Boxplot of Population Features', fontdict={'size': 18, 'weight': 'bold'});

*Observations on Demographics Features*

The boxplots show some values for some of the features as outliers. As an example, Total Population has some very notable and large outliers, one of which happens to be Tempe, the other of which is 80219. The latter has a population density, however, that isn't an outlier, which indicates that the population is perhaps a function of another variable, possibly the total land area of the zip code. Similarly, the outliers in population density are both very small counties in Denver and suggest that perhaps there are a higher number of multi-family housing units in those zip codes. Please note that I looked for data that had a breakdown of the types of housing units in each zip code, but could find no reliable source. I conclude from what I've seen and the research I've done that some of the features I'm using are likely affected by other features that I cannot reliably collect or determine.

That being said, I do not believe I can qualify any of the values in the dataset as true 'outliers'. In a sense, the values are what they are and are intrinsic measurements of a specific area. K-Means clustering is sensitive to outliers, so I need to choose a method of scaling the data that will appropriately represent the outliers. `Scikit-learn` includes a preprocessor named `RobustScaler` which removes the median and scales the data according to the quantile range and is therefore not influenced by a few number of very large marginal outliers. I will use this preprocessor to scale the data prior to clustering.

Next, I examine the correlation matrix of the entire dataset to see if we can find any patterns that might help with further dimensionality reduction.

In [ ]:
# For most of the rest of the exploration and clustering we need this data without
# zip code, latitude or longitude
df_clustering = final_merged.drop(['ZipCode','Latitude','Longitude'], axis=1)
corr = df_clustering.corr()

# Create a correlation matrix on our demograpics data, but without coordinates
corr = df_clustering.corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(16,9))

# Generate a custom diverging colormap
cmap = sns.color_palette("BrBG")

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .75});
ax.set_title('Correlation Matrix', fontdict={'size': 14, 'weight': 'bold'});

*Observations on Correlation Matrix*

From the matrix, there are some correlations that make sense. For home values, the average and median values are strongly positively correlated, as are median/average home values and average household income. Households and housing units are obviously strongly correlated because, while a housing unit is a house, an apartment, a mobile home, a group of rooms, or a single room occupied (or if vacant, intended for occupancy) as separate living quarters, a household includes all the people who occupy a housing unit. So, these two things would vary depending on just sheer numbers of housing units.

A couple of interesting correlations are also revealed. Average home value is strongly negatively correlated to both diversity index and average household size, which means larger households occupy lower-valued homes and that neighborhoods containing a high-degree of a single ethnicity (regardless of the ethnicity) contain lower-valued homes. Similarly, per capita income is also strongly negatively correlated with diversity index, meaning that more diverse neighborhoods have lower incomes than less diverse ones. Lastly, the venue category Nightlife Spot was fairly strongly negatively correlated with Population in Families, possibly suggesting that families don't choose to live near nightlife spots.

## Dimensionality Reduction

Based based on the correlations in the matrix and an understanding of the underlying data, I am going to be removing several features. Total population is a good candidate for removal, since we have a similar and more rich feature, Population Density that is more descriptive because it takes into account the size of a zip code, which is data I cannot obtain. Since I have made a modification to Median Home Value and Average Home Value is strongly correlated to that, I'm removing Average Home Value. I'm also removing the features related to housing units, because we have a similar measure, Total Households, to which it is also strongly positively correlated. I'm also removing one feature from the venues set, 'Residence', because I do not consider Foursquare to be a good source of residence information.

The resulting DataFrame contains 85 observations with 23 features.

In [ ]:
# Remove the noted features
df_clustering.drop(['Average Home Value','Total Population','Total Housing Units', 'Vacant Housing Units', 'Residence'], 
                   axis=1, inplace=True)

## Identifying Candidate Zip Codes

We will use our reduced-dimension dataset to cluster all the data to find any Denver metro zip codes that are in the same cluster as Tempe. We first scale the data using RobustScaler and then attempt to find an optimum number of clusters using the 'elbow' methodology.

In [ ]:
# Here we rescale the data with RobustScaler
from sklearn.preprocessing import RobustScaler
from sklearn.cluster import KMeans

# Do not rescale zip code, latitude or longitude
features = [c for c in df_clustering if c not in ['ZipCode','Latitude', 'Longitude']]
X = df_clustering.loc[:, features].values
rescaled = RobustScaler().fit_transform(X)

# Sum of squared distances, for the elbow method
ssd = []

# Range of values for k
K = range(2,20)

for k in K:
    # run k-means clustering
    kmeans = KMeans(n_clusters=k, n_init=100, random_state=13031)
    kmeans = kmeans.fit(X)
    ssd.append(kmeans.inertia_)

# Generate a basic plot to help us decide on a value of k
fig, ax = plt.subplots(figsize=(16,9))
sns.lineplot(x=K, y=ssd)
ax.set_title('Elbow Evaluation For Optimal k', fontdict={'size': 14, 'weight': 'bold'})
ax.set(xticks=np.arange(2,20))
ax.set_xlabel('k', fontdict={'size': 14})
ax.set_ylabel('Sum of Squared Distances', fontdict={'size': 12});

Based on the results there is a distinct elbow around 6 or 7 and another at about 16. For this analysis, I'm choosing 7 as the number of clusters to build from the dataset.

In [ ]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# Run k-means clustering with the optimized values and assign the cluster labels to our original dataset.
kclusters = 7
kmeans = KMeans(n_clusters=kclusters, random_state=13031)
kmeans = kmeans.fit(X)

# Add clustering labels directly to the final merged set because nothing changed the index
try:
    final_merged.drop(['Cluster Labels'], axis=1, inplace=True)
except:
    pass
final_merged.insert(0, 'Cluster Labels', kmeans.labels_)

# We also need the cluster labels added to the one-hot-encoding-derived top 10 list. That
# was indexed differently so we use merge to add those
fm_labels = final_merged.loc[:, ['ZipCode','Cluster Labels']].copy()
clustered_top10 = pd.merge(fm_labels, result_venues_sorted, left_on='ZipCode', right_on='ZipCode', how='inner', validate='1:1')
clustered_top10 = clustered_top10[['Cluster Labels','ZipCode'] + [c for c in clustered_top10 if c not in ['Cluster Labels','ZipCode']]];

In [ ]:
# Create cluster map
map_clusters = folium.Map(location=denxy, zoom_start=10)

# Set color scheme for the clusters. We are using the tab10 qualitative colormap
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.tab10(np.linspace(0, 1, len(ys)))
tab10 = [colors.rgb2hex(i) for i in colors_array]

# Add markers to the map
markers_colors = []
for zipcode, lat, lon, cluster in zip(final_merged['ZipCode'], final_merged['Latitude'], final_merged['Longitude'], final_merged['Cluster Labels']):
    label = folium.Popup('{} - Cluster {}'.format(zipcode, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=tab10[cluster],
        fill=True,
        fill_color=tab10[cluster],
        fill_opacity=0.5).add_to(map_clusters)

# Use the utility library to save the map as a png
csutil.save_map(map_clusters, html_fn='cluster_map.html', png_fn='./maps/cluster_map.png');

Now that clustering is complete, wecan visualize the zip code clusters across the Denver metro area.

In [ ]:
# Display the clusters
map_clusters

# Results

From the seven clusters generated, Tempe was placed into Cluster 6. With this in mind, we can now start to examine the cluster and the data we have to see if we can find from the candidates any suitable locations to pursue for my expansion. Below is a brief summary of the top venues in each of the zip codes in Cluster 6.

In [ ]:
clustered_top10.loc[clustered_top10['Cluster Labels'] == 6, 
                    ['ZipCode', '1st Most Common Venue', '2nd Most Common Venue','3rd Most Common Venue']]

This shows that for all but one zip code in the cluster, a Coffee Shop is in the top 3 venues recommended, and for that one where it isn't, that a Coffee Shop is still in the top 5. So, people are going to these zip codes for coffee, which is at least promising. Lets take a look at the demographics to see how those line up.

In [ ]:
# Filter from the full results our Cluster 6 and show an example of the 
candidates = final_merged.loc[final_merged['Cluster Labels'] == 6, :]
candidates.loc[:, ['ZipCode', 'Median Home Value', 'Diversity Index3','Food', 'Shop & Service']]

### A Look at Existing Coffee Shops

Lastly, I want to dig a little into the existing coffee shops in the candidate zip codes, see what types they are (meaning are they franchise or independent) and where they are located. I'm also only concerned with actual coffee shops, so because the Foursquare 'coffee' section returns things like tea rooms, cafés, and convenience stores (all of whom serve coffee), those venues aren't really like my shop. I also wanted to distinguish between franchise and independent coffee shops. We retrieve the top 100 recommended venues in the 'coffee' category, remove the venues that are not coffee shops, and determine whether each is a franchise or not using a list of coffee franchises obtained from Wikipedia.

The following summarizes the results of this analysis.

In [ ]:
# Get the shops from the candidates using Foursquare EXPLORE with a section = 'coffee'
pivoted_shops = csutil.load_coffee_shops(names=candidates['ZipCode'],
                                        lats=candidates['Latitude'],
                                        lngs=candidates['Longitude'])

# Draw the plot
fig, ax = plt.subplots(figsize=(12,6))
bp_all = sns.barplot(x='ZipCode', y='All', data=pivoted_shops, color='royalblue', label='All Coffee Shops')
sns.barplot(x='ZipCode', y='False', data=pivoted_shops, color='cornflowerblue', label='Independent Coffee Shops')
ax.set_title("Percent Independent Coffee Shops in Candidate Zip Codes", fontdict={'size': 14, 'weight': 'bold'})
ax.set_xlabel('Zip Code', fontdict={'size': 12})
ax.set_ylabel('Count', fontdict={'size': 12})
ax.legend(loc='best');

# Label the bars with percent independent
for idx, rect in enumerate(ax.patches[:10]):
    ax.text(rect.get_x() + rect.get_width()/2, 
            rect.get_height() + .006, 
            str(int((pivoted_shops.loc[idx, 'PctInd'] * 100).round(0))) + '%',
            ha='center', va='bottom', fontdict={'fontsize': 12});

# Discussion

The results of the clustering revealed nine other zip codes that were similar in nature to Tempe, and while each of these zip codes might warrant further investigation I'd like to choose a small set of them to focus on, using our stated criteria.

## Deeper Look at Candidates

### Exclusions

Of the nine other zip codes in the cluster, the following are being excluded from further consideration: 80003, 80013, 80014, 80214, and 80249. Zip code 80013 has no independent coffee shops, but the most commonly visited venue there is a convenience store, which I assume is a top venue for coffee. Zip code 80249 is near Denver International Airport. Zip codes 80003, 80214, and 80204 already have more than 50% independent coffee shops. Further, based on other investigation, many contain no 'anchor' venues, and despite some of their proximity to parks simply do not meet the stated criteria.

### Final Candidate Zip Codes

The following zip codes show promise as possible expansion zip codes. Further analysis would be needed to determine overall feasibility which should include appropriate and available real estate (including proximity to any existing coffee shops), further demographic analysis to include breakdowns of age groups and ethnic diversity, crime rates and other relevant criteria. 

**80204**

This area is just to the west of downtown Denver with close proximity to popular venues such as the Colorado Convention Center, Denver Center for the Performing Arts, and the trendy LoDo neighborhood. It also includes both Mile High stadium and the Pepsi Center, which host Denver's professional sports teams and A-List concerts. It encompasses both the Auraria Campus and the West Colfax and Lincoln Park neighborhoods. It is also host to the Santa Fe Arts District and has good proximity to Sloan's Lake Park. One shortcoming is that it already contains a relatively high number of independent coffee shops, approximately 53%. Coffee shops are also the third most common venue visited in this zip code.

**80236**

This is largely a residential neighborhood that contains Mullen High School, a prestigious private Catholic preparatory school. It is comprised the Harvey Park South neighborhood, which contains homes built in the 1950s, like Tempe. [Charms and challenges in Denver’s best kept “mid-mod” secret](https://www.denverpost.com/2012/09/28/charms-and-challenges-in-denvers-best-kept-mid-mod-secret/) describes some of these homes by saying, "That these are special homes was far from secret in 1955. "*This design was so well-thought-out that several of the buyers were young architects who had been trained after World War II on the G.I. Bill*" It is also about a ten minute drive to the University of Denver and has close proximity to Marsten Lake, which is a popular birding location. It has the third lowest percentages of independent coffee shops, at about 38%. Coffee shops are the second most common venue visited in this zip code.

**80110**

This area is a mix of commercial and residential almost directly south of downtown Denver. It has very close proximity to Cherry Hills Village, which is a very affluent neighborhood with a very prestigious golf club. It also has good proximity to Swedish Medical Center and Craig Hospital in Englewood as well as to the Gothic Theater, which is an up-close live music venue converted from an art deco movie house. It also has close proximity to Arapahoe Community College in Littleton and the University of Denver. It has the second lowest percentages of independent coffee shops, at about 36%. Coffee shops are the second most common venue visited in this zip code.

# Conclusion

In this study I sought to identify some target zip codes in the Denver metro area that could potentially be a site for the expansion of my coffee shop in Tempe, AZ. I combined demographic data for selected zip codes in the Denver metro area with venue and category data from Foursquare and used the data as the basis for clustering to identify zip codes that are most similar to 85281. Based on the clustering nine candidate zip codes were found, of which three were identified as the most suitable for further investigation and planning. Before considering any expansion those three would need further, detailed analysis to identify a suitable location. 

# References

The following sources were used in the creation of this study.

* IBM Applied Data Science Course Materials and Labs
* [**My Python Utility Module**](https://github.com/jrfoster/Coursera_Capstone/blob/master/capstoneutils.py)
* [Interactive Cluster Map]()
* [Sperling's Best Places](https://www.bestplaces.net/cost-of-living/tempe-az/denver-co/50000)
* [Foursquare API Documentation](https://developer.foursquare.com/docs/api/endpoints)
* [scikit-learn Documentation](https://scikit-learn.org/stable/index.html)
* [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/)
* [Anytree Documentation](https://anytree.readthedocs.io/en/latest/)
* [Simpsons Diversity Index](http://www.countrysideinfo.co.uk/simpsons.htm)
* [Google Geocoding API](https://developers.google.com/maps/documentation/geocoding/start)
* [Top Neighborhoods for Young Professionals](http://urbanphenix.com/neighborhoods-in-denver-for-young-professionals/)
* [Charms and challenges in Denver’s best kept “mid-mod” secret](https://www.denverpost.com/2012/09/28/charms-and-challenges-in-denvers-best-kept-mid-mod-secret/)
* [List of Coffeehouse Chains](https://en.wikipedia.org/wiki/List_of_coffeehouse_chains)
